# Example notebook

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pertpy
from deres import DEResult
import scanpy as sc
import numpy as np
import pandas as pd

/home/sturm/projects/scverse/deres/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
adata = sc.datasets.pbmc3k_processed()

In [4]:
adata.obs["sample"] = pd.Categorical(np.random.randint(0, 10, adata.shape[0]))
adata = sc.AnnData(adata.raw.X, obs=adata.obs, var=adata.raw.var)

In [5]:
adata.obs

,n_genes,percent_mito,n_counts,louvain,sample
index,,,,,
AAACATACAACCAC-1,781,0.030178,2419.0,CD4 T cells,0
AAACATTGAGCTAC-1,1352,0.037936,4903.0,B cells,7
AAACATTGATCAGC-1,1131,0.008897,3147.0,CD4 T cells,6
AAACCGTGCTTCCG-1,960,0.017431,2639.0,CD14+ Monocytes,3
AAACCGTGTATGCG-1,522,0.012245,980.0,NK cells,1
...,...,...,...,...,...
TTTCGAACTCTCAT-1,1155,0.021104,3459.0,CD14+ Monocytes,4
TTTCTACTGAGGCA-1,1227,0.009294,3443.0,B cells,8
TTTCTACTTCCTCG-1,622,0.021971,1684.0,B cells,1


In [6]:
pb = sc.get.aggregate(adata, ["sample", "louvain"], "mean")

In [7]:
adata

AnnData object with n_obs × n_vars = 2638 × 13714
    obs: 'n_genes', 'percent_mito', 'n_counts', 'louvain', 'sample'
    var: 'n_cells'

In [8]:
pb.layers["mean"]

array([[0.        , 0.00722028, 0.        , ..., 0.00722028, 0.03310473,
        0.00722028],
       [0.        , 0.        , 0.        , ..., 0.01414586, 0.01414586,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.02100446, 0.        ,
        0.04200892],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [9]:
mod = pertpy.tl.Statsmodels(pb, design="~ louvain", layer="mean")
mod.fit()

100%|██████████| 13714/13714 [00:11<00:00, 1185.96it/s]


In [10]:
res = mod.test_contrasts({"CD8_vs_CD4": mod.cond(louvain="CD8 T cells") - mod.cond(louvain="CD4 T cells")})

100%|██████████| 13714/13714 [00:01<00:00, 10685.93it/s]


In [11]:
res

,variable,p_value,t_value,sd,log_fc,adj_p_value,contrast
13001,NKG7,1.6818550715162358e-31,2.065608e+01,0.084947,1.754663e+00,1.681855e-31,CD8_vs_CD4
11083,CCL5,1.5679378087397538e-30,1.990107e+01,0.091021,1.811422e+00,1.567938e-30,CD8_vs_CD4
11879,CST7,8.852244689571697e-27,1.715730e+01,0.050831,8.721260e-01,8.852245e-27,CD8_vs_CD4
3695,GZMA,4.320840028518062e-25,1.600597e+01,0.051367,8.221842e-01,4.320840e-25,CD8_vs_CD4
9292,GZMH,5.101918523190993e-23,1.465539e+01,0.040967,6.003942e-01,5.101919e-23,CD8_vs_CD4
...,...,...,...,...,...,...,...
8100,COLCA2,1.0,3.770236e-17,0.001420,5.355299e-20,1.000000e+00,CD8_vs_CD4
5992,GFRA2,1.0,-6.735480e-17,0.009428,-6.350516e-19,1.000000e+00,CD8_vs_CD4
1891,AC104655.3,1.0,2.738473e-17,0.001952,5.346155e-20,1.000000e+00,CD8_vs_CD4
12993,KLK1,1.0,5.772808e-17,0.005040,2.909340e-19,1.000000e+00,CD8_vs_CD4


In [12]:
de_res = DEResult(res, adata, p_col="p_value", effect_size_col="log_fc", var_col="variable")

In [13]:
de_res

In [17]:
de_res.summary()

,total,up,down
p < 0.1,1037,394,643
p < 0.05,736,259,477
p < 0.01,437,133,304
p < 0.001,265,65,200
p < 0.0001,153,48,105
